# Less-Referenced Mosaic

This notebook provides a cross-section of the Less-Referenced Mosaic creation process.

# Setup

## Imports

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state
import yaml

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, preprocess, mosaic, raster, pipelines

## Settings

In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
local_settings = {
    'mosaic_filepath': 'mosaics/referenced3.tiff',
    'overwrite': True,
    'random_state': 16849,
    'train_size': 1,
}
settings.update(local_settings)

## Parse Settings

In [ ]:
settings['mosaic_filepath'] = os.path.join(settings['data_dir'], settings['mosaic_filepath'])

In [ ]:
for key, relpath in settings['paths_relative_to_data_dir'].items():
    settings[key] = os.path.join(settings['data_dir'], relpath)

In [ ]:
random_state = check_random_state(settings['random_state'])

In [ ]:
referenced_fps = utils.discover_data(settings['referenced_images_dir'], ['tif', 'tiff'], pattern=r'Geo\s\d+_\d.tif')

In [ ]:
palette = sns.color_palette(settings['color_palette'])

# Prepare Data
The first part is to prepare the data (AKA extract/transform/load).

## Train-Test Split

We split the data into training data (data that is georeferenced) and test data (data that is not georeferenced, or for which we don't use the georeferencing information when we're building the models).

We set the train size to some small number, because ideally the user only needs to georeference a couple of images manually.

In [ ]:
settings['train_size']

In [ ]:
fps_train, fps_test = train_test_split(referenced_fps, train_size=settings['train_size'], random_state=settings['random_state'])

## Initial, Approximate Georeferencing
We use the sensor (high-altitude balloon) positions to provide approximate georeferencing, which will be useful for saving computational time when building the unreferenced mosaic.

In [ ]:
# Get the pipeline(s)
sensor_georeference_pipeline_X, sensor_georeference_pipeline_y = pipelines.GeoreferencePipelines.sensor_georeference()
sensor_georeference_pipeline_X.set_params(nitelite__output_columns=['filepath', 'sensor_x', 'sensor_y'], sensor_georeference__passthrough=['filepath'])

In [ ]:
# Get the geo-transforms used for training
geotransforms_train = sensor_georeference_pipeline_y.fit_transform(fps_train)

In [ ]:
# Train the pipeline
sensor_georeference_pipeline_X.fit(
    fps_train,
    geotransforms_train,
    nitelite__img_log_fp=settings['img_log_fp'],
    nitelite__imu_log_fp=settings['imu_log_fp'],
    nitelite__gps_log_fp=settings['gps_log_fp'],
)

In [ ]:
# The fit gives us an estimate of the error from the approximate georeferencing
sensor_georeference_pipeline_X.score(fps_train, geotransforms_train)

In [ ]:
first_pass_error = sensor_georeference_pipeline_X.named_steps['sensor_georeference'].spatial_error_

In [ ]:
# Get the approximate georeferences of the test filepaths
X_test = sensor_georeference_pipeline_X.predict(fps_test)

## Create the Containing Mosaic Dataset

In [ ]:
# DEBUG
import importlib
importlib.reload(mosaic)

In [ ]:
if os.path.isfile(settings['mosaic_filepath']) and settings['overwrite']:
    os.remove(settings['mosaic_filepath'])

In [ ]:
mos = mosaic.Mosaic(settings['mosaic_filepath'])

In [ ]:
mos.fit(X_test[['filepath'] + preprocess.GEOTRANSFORM_COLS])

In [ ]:
# Close the dataset
mos.dataset_.FlushCache()
mos.dataset_ = None

## Create the Starting Image for the Mosaic

In [ ]:
reffed_mosaic_pipeline = pipelines.MosaicPipelines.referenced_mosaic(settings['mosaic_filepath'])
reffed_mosaic_pipeline.set_params(mosaic__exist_ok=True)

In [ ]:
# Make the referenced mosaic
reffed_mosaic_pipeline.fit_transform(fps_train)

## The Less-Referenced Mosaic

In [ ]:
less_reffed_mosaic = mosaic.LessReferencedMosaic(filepath=settings['mosaic_filepath'], padding=padding)

In [ ]:
# First, the fit
less_reffed_mosaic.fit(y_train)

In [ ]:
# Then create the referenced mosaic
less_reffed_mosaic.predict(X_test[['filepath'] + preprocess.GEOTRANSFORM_COLS])

# Inspect Results

In [ ]:
# DEBUG
importlib.reload(utils)

In [ ]:
full_img = less_reffed_mosaic.dataset_.ReadAsArray().transpose(1, 2, 0)

In [ ]:
# One of the images
iter_inds = less_reffed_mosaic.log['iteration_indices']
X_iter = X_test.loc[iter_inds]

In [ ]:
i = 0
row = X_iter.iloc[i]

In [ ]:
# Expected bounds
x_min = row['x_min'] - less_reffed_mosaic.padding
x_max = row['x_max'] + less_reffed_mosaic.padding
y_min = row['y_min'] - less_reffed_mosaic.padding
y_max = row['y_max'] + less_reffed_mosaic.padding

In [ ]:
# Rectangle of the image to add to the plot

# Convert to offset and size
x_off, y_off, x_size, y_size = less_reffed_mosaic.bounds_to_offset(x_min, x_max, y_min, y_max)

rect = patches.Rectangle(
    (x_off, y_off),
    x_size,
    y_size,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.gca()

# Image itself
ax.imshow(full_img)

# The first image location
ax.add_patch(rect)

In [ ]:
# The images
dst_img = less_reffed_mosaic.get_image(x_min, x_max, y_min, y_max)
src_img = utils.load_image(
    row['filepath'],
    dtype=less_reffed_mosaic.dtype,
)

In [ ]:
# The existing mosaic we loaded
plt.imshow(dst_img)

In [ ]:
# The image we're adding
plt.imshow(src_img)

In [ ]:
M, mask = utils.calc_warp_transform(src_img, dst_img)

In [ ]:
assert utils.validate_warp_transform(M)

In [ ]:
settings['mosaic_filepath']

In [ ]:
less_reffed_mosaic.dataset_.FlushCache()